In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
search = TavilySearchResults()

In [3]:
search.invoke("what is the weather is SF")

[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA',
  'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 3:02 pm PDT Mar 14, 2024. Forecast Valid: 8pm PDT Mar 14, 2024-6pm PDT Mar 21, 2024 .'},
 {'url': 'https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US',
  'content': "Today's and tonight's San Francisco, CA weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com"},
 {'url': 'https://weather.com/weather/hourbyhour/l/USCA0987:1:US',
  'content': "recents\nSpecialty Forecasts\nHourly Weather-San Francisco, CA\nBeach Hazard Statement\nSaturday, November 25\n5 pm\nClear\n6 pm\nClear\n7 pm\nClear\n8 pm\nMostly Clear\n9 pm\nPartly Cloudy\n10 pm\nPartly Cloudy\n11 pm\nPartly Cloudy\nSunday, Novem

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [7]:
tools = [get_word_length]

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [9]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
agent_executor.invoke({"input":"How many letters in the word educa?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'How many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}